In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import re
import time
from bs4.element import Comment
import time, sys
from IPython.display import clear_output
import urllib
import os
import bs4
import utils
import numpy as np

def update_progress(cur, total):
    progress = cur/total
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress:{2} of {3} [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100,cur,total)
    print(text)


In [2]:
# save a text file
def save_textFile(obj, path):	
	os.makedirs(os.path.dirname(path), exist_ok=True)
	with open(path, "w", encoding='utf-8', errors='ignore') as f:
		f.write(obj)
# load a text file
def load_textFile(name):
	text_file = open('data/' + name, 'r', encoding='utf-8', errors='ignore')
	mytext = text_file.read()
	text_file.close()
	
	return mytext

def write_list(l, filename):
	filename = add_txt_ext(filename)
	if os.path.dirname(filename)!='':
		os.makedirs(os.path.dirname(filename), exist_ok=True)
	with open(filename,'w',encoding='utf-8') as f:
		for item in l:
			f.write(str(item)+'\n')
			
def load_list(filename): 
	out = []
	with open(filename,'r', encoding='utf-8',errors='ignore') as f:
		l = f.readline()
		while l!="":
			l = l.strip()#" ".join(re.split("\s+",l))
			out.append(l)
			l = f.readline()
	return out

def append_to_file(val, filename):
	filename = add_txt_ext(filename)
	if os.path.dirname(filename)!='':
		os.makedirs(os.path.dirname(filename), exist_ok=True)
	with open(filename, 'a') as f:
		if type(val)==type(list()):
			for v in val:
				f.write(str(v)+"\n")
		else:
			f.write(str(val)+"\n")

def add_txt_ext(filename):
	path,ext = os.path.splitext(filename)
	if ext != "txt":
		filename = path+".txt"
	return filename
	
def remove_class(soup, class_):
	found = True
	while found:
		found = remove_class_h(soup, class_)

def remove_class_h(soup, class_):		
	
	for desc in soup.descendants:
		if type(desc) == bs4.element.Tag and 'class' in desc.attrs.keys():							
			for c in desc['class']:
				if class_ in c:
					desc.decompose()
					return True
	return False

def remove_tag(soup, tag):
	for script in soup([tag]):
		script.decompose()

In [12]:
df = pd.DataFrame(columns=['url','doc_string'])

In [3]:

page = requests.get(url)

allpagesStrain = SoupStrainer(class_="allpageslist")
allpagesSoup = BeautifulSoup(page.content, 'html.parser', parse_only=allpagesStrain)
allpagesList = []
# add page directories to a list
for link in allpagesSoup.find_all('a'):
    linkString = link.get('href')
    if len(allpagesList)>0 and allpagesList[-1] == linkString:
        continue
    else: 
        allpagesList.append(linkString)


In [4]:
pageList = []
idx = 0
for link in allpagesList:
    print("parsing link {} of {}".format(idx,len(allpagesList)-1 ))
    # request page
    url = baseurl + link
    page = requests.get(url)
    
    # filter for table content
    pageStrain = SoupStrainer(class_="mw-allpages-table-chunk")
    pagesSoup = BeautifulSoup(page.content, 'html.parser', parse_only=pageStrain)
    
    # add the pages to a list
    for link in pagesSoup.find_all('a'):
        linkString = link.get('href')
        if not "Legends_of_Runeterra" in linkString:
            pageList.append(linkString)
    
    time.sleep(2)
    idx += 1


parsing link 0 of 26
parsing link 1 of 26
parsing link 2 of 26
parsing link 3 of 26
parsing link 4 of 26
parsing link 5 of 26
parsing link 6 of 26
parsing link 7 of 26
parsing link 8 of 26
parsing link 9 of 26
parsing link 10 of 26
parsing link 11 of 26
parsing link 12 of 26
parsing link 13 of 26
parsing link 14 of 26
parsing link 15 of 26
parsing link 16 of 26
parsing link 17 of 26
parsing link 18 of 26
parsing link 19 of 26
parsing link 20 of 26
parsing link 21 of 26
parsing link 22 of 26
parsing link 23 of 26
parsing link 24 of 26
parsing link 25 of 26
parsing link 26 of 26


In [5]:
cleanpageList = []
for item in pageList:
    temp = item.split('/wiki/')
    cleanpageList.append(temp[1])


In [6]:
# get parent pages
parentPages = []
for item in cleanpageList:
    if '/' in item:
        temp = item.split('/')
        parentPages.append(temp[0])
parentPages = list(set(parentPages))

# remove parent pages from page list and sort alphabetically
cleanpageList = list(set(cleanpageList) - set(parentPages))
cleanpageList.sort()

In [19]:
filename = "data/LeagueFandom.txt"
with open(filename,'w') as f:
    for l in cleanpageList:
        f.write(str(l)+'\n')

In [2]:
filename = "data/LeagueFandom.txt"
pageList = []
with open(filename,'r') as f:
    l = f.readline()
    while l!="":
        pageList.append(l[:-1])
        l = f.readline()
    

In [66]:
files = ["data/mobafire_CompletedPagesList.txt"]
pages = load_list(files[0])
terms = set()
for p in pages:
    words = p.split('/')
    for w in words:
        if '-' in w:
            # replace '-' with ' ' 
            w = w.split('-')
            # remove the last part if it is a number or "guide"
            if w[-1] == 'guide' or w[-1].isnumeric():
                w = w[:-1]
            w = " ".join(w)
        terms.add(w.lower())
mobafireTermsList = list(terms)
mobafireTermsList.sort()        

In [67]:
file = "data/LeagueFandom_CompletedPagesList.txt"
pages = load_list(file)
terms = set()
for p in pages:
    if "Teamfight_Tactics" in p:
        continue
    words = p.split('/')
    for w in words:        
        w = w.replace('_',' ')            
        w = w.replace('%27','\'')
        terms.add(w.lower())
fandomTermsList = list(terms)
fandomTermsList.sort()   

In [68]:
file = "data/gamepedia_CompletedPagesList.txt"
pages = load_list(file)
terms = set()
for p in pages:
    if "Teamfight_Tactics" in p:
        continue
    words = p.split('/')
    for w in words:        
        w = w.replace('_',' ')            
        w = w.replace('%27','\'')
        terms.add(w.lower())
gamepediaTermsList = list(terms)
gamepediaTermsList.sort()   

In [53]:
gamepediaTermsList

['Aatrox',
 'Absolute Focus (Rune)',
 'Abyssal Mask (Item)',
 'Abyssal Scepter (Item)',
 'Adaptive Helm (Item)',
 'Aegis of the Legion (Item)',
 'Aether Wisp (Item)',
 'Aftershock (Rune)',
 'Ahri',
 'Akali',
 'Alacrity (Item)',
 'Alistar',
 'Alphabetically',
 'Amplifying Tome (Item)',
 'Amumu',
 'Ancient Coin (Item)',
 'Ancient Golem',
 'Ancient Krug',
 'Ancient Pocket Watch (Item)',
 'Anivia',
 'Annie',
 'Anti-Turret Cannon',
 'Approach Velocity (Rune)',
 'Arcane Comet (Rune)',
 'Arcane Helix (Item)',
 'Arcane Sweeper (Item)',
 'Arcane Sweeper (Trinket) (Item)',
 "Archangel's Staff (Item)",
 'Ardent Censer (Item)',
 'Ascension (Game Mode)',
 'Ashe',
 'Aspect of the Dragon',
 "Athene's Unholy Grail (Item)",
 "Atma's Impaler (Item)",
 "Atma's Reckoning (Item)",
 'Aurelion Sol',
 'Avarice Blade (Item)',
 'Azir',
 'B.F. Sword (Item)',
 'Bag of Tea (Item)',
 "Bami's Cinder (Item)",
 'Banner of Command (Item)',
 "Banshee's Veil (Item)",
 'Bard',
 'Baron Nashor',
 'Barrier',
 "Berserker's Gr

In [69]:
terms = set.union(set(mobafireTermsList), set(fandomTermsList),set(gamepediaTermsList))
termsList = list(terms)
termsList.sort()
write_list(termsList, "data/termsList.txt")

In [46]:
pageListFilename = "data/gamepedia.txt"
baseurl = "https://lol.gamepedia.com"

navStrainer = SoupStrainer(class_="mw-allpages-nav")
bodyStrainer = SoupStrainer(class_="mw-allpages-body")



page = baseurl + "Special:AllPages"

if os.path.exists(pageListFilename):
    with open(pageListFilename, 'rb') as f:
        f.seek(-2, os.SEEK_END)
        while f.read(1) != b'\n':
            f.seek(-2, os.SEEK_CUR)
        last_line = f.readline().decode()
        page = "/index.php?title=Special:AllPages&from="+last_line[1:].strip()
        print("starting at: ", baseurl+page)
        time.sleep(5)

while not page=="":
    clear_output(wait = True)
    print("At page: {}".format(baseurl + page))    
    req = requests.get(baseurl+page)
    nav  = BeautifulSoup(req.content, "lxml", parse_only=navStrainer)
    body = BeautifulSoup(req.content, "lxml", parse_only=bodyStrainer)
    
    # find the "next page" link and add it to the list
    # but only take the first one (there are at least 2 on the page)    
    page = ""
    for tag in nav.find_all("a"):
        if "Next page" in tag.string:          
            page = tag['href']            
            curPageTitle = tag.string.replace("Next page","")
            break
    pageList = list()
    for tag in body.find_all("a"):
        pageList.append(tag['href'])
        
    append_to_file(pageList,pageListFilename)
    time.sleep(1)
        
            
    
    
print("finished")
    



At page: https://lol.gamepedia.com/index.php?title=Special:AllPages&from=%ED%99%A9%EC%9E%AC%ED%9B%88
finished


<!DOCTYPE html>



In [3]:
navStrainer = SoupStrainer(class_="mw-allpages-nav")
bodyStrainer = SoupStrainer(class_="mw-allpages-body")
page = "https://lol.gamepedia.com/Special:AllPages"

req = requests.get(page)
nav  = BeautifulSoup(req.content, "lxml", parse_only=navStrainer)
body = BeautifulSoup(req.content, "lxml", parse_only=bodyStrainer)





In [5]:
for tag in nav.find_all("a"):
    print(tag['href'])

/index.php?title=Special:AllPages&from=2013+GPL+Summer%2FVoDs
/index.php?title=Special:AllPages&from=2013+GPL+Summer%2FVoDs


In [16]:
len(pageList)

2070

In [4]:
portals = ['/New_To_League/Welcome',
           '/Portal:Champions',
           '/Portal:Items',
           '/Portal:Minions',
           '/Portal:Monsters',
           '/Portal:Summoner_Spells',
           '/Portal:Runes',
           '/Portal:Patch_Notes',
           '/Portal:Game_Modes',
           '/Portal:Miscellaneous']

baseurl = "https://lol.gamepedia.com"
pageList = list()
for p in portals:
    print(baseurl+p)
    resp = requests.get(baseurl+p)
    strainer = SoupStrainer(class_=["tabheader-tab",'mw-body-content'])
    soup = BeautifulSoup(resp.content,'lxml',parse_only=strainer)
    l = list()
    soup.find('div', id="top-schedule").decompose()
    for t in soup.find_all("a"):    
        if 'href' in t.attrs and "http" not in t['href'] and t['href'][0] != "#" and "action=edit" not in t['href']:
            l.append(t['href'])       
    
    for a in l:
        pageList.append(a)
        

    
    



https://lol.gamepedia.com/New_To_League/Welcome
https://lol.gamepedia.com/Portal:Champions
https://lol.gamepedia.com/Portal:Items
https://lol.gamepedia.com/Portal:Minions
https://lol.gamepedia.com/Portal:Monsters
https://lol.gamepedia.com/Portal:Summoner_Spells
https://lol.gamepedia.com/Portal:Runes
https://lol.gamepedia.com/Portal:Patch_Notes
https://lol.gamepedia.com/Portal:Game_Modes
https://lol.gamepedia.com/Portal:Miscellaneous


In [10]:
pageList = list(set(pageList))
pageList.sort()
remove = ["/Patch", "Portal:", "/2018", "File:","Bjergsen","TSM","Echo_Fox","#","Template:"]
for r in remove:
    pageList = [p for p in pageList if r not in p]
for p in pageList:
    print(p)

In [96]:
write_list(pageList, "data/gamepedia.txt")

In [109]:
len(pageList)

759

/Aatrox
/Absolute_Focus_(Rune)
/Abyssal_Mask_(Item)
/Abyssal_Scepter_(Item)
/Adaptive_Helm_(Item)
/Aegis_of_the_Legion_(Item)
/Aether_Wisp_(Item)
/Aftershock_(Rune)
/Ahri
/Akali
/Alistar
/All_Random_Summoner%27s_Rift
/Amplifying_Tome_(Item)
/Amumu
/Ancient_Coin_(Item)
/Ancient_Golem
/Ancient_Krug
/Ancient_Pocket_Watch_(Item)
/Anivia
/Annie
/Anti-Turret_Cannon
/Aphelios
/Approach_Velocity_(Rune)
/Arcane_Comet_(Rune)
/Arcane_Helix_(Item)
/Arcane_Sweeper_(Item)
/Arcane_Sweeper_(Trinket)_(Item)
/Arcanist_Kog%27Maw
/Arcanist_Shaco
/Arcanist_Zoe
/Arcanist_Zoe_Prestige_Edition
/Archangel%27s_Staff_(Item)
/Ardent_Censer_(Item)
/Ascension_(Game_Mode)
/Ashe
/Aspect_of_the_Dragon
/Athene%27s_Unholy_Grail_(Item)
/Atma%27s_Impaler_(Item)
/Atma%27s_Reckoning_(Item)
/Aurelion_Sol
/Avarice_Blade_(Item)
/Azir
/B.F._Sword_(Item)
/Bag_of_Tea_(Item)
/Bami%27s_Cinder_(Item)
/Banner_of_Command_(Item)
/Banshee%27s_Veil_(Item)
/Bard
/Baron_Nashor
/Barrier
/Barrier_(Summoner_Spell)
/Berserker%27s_Greaves_(Item

In [6]:
# collect bold and header terms
sites=['gamepedia','LeagueFandom','mobafire']

for site in sites:    
    terms = list()
    #site = sites[0]
    folder = os.path.join("data",site+"Data")
    files = os.listdir(folder)
    boldFiles = [b for b in files if "boldTerms" in b]
    headerFiles = [h for h in files if "headerTerms" in h]
    fileLists = [boldFiles, headerFiles]
    termTypeNames = ["bold", "header"]
    for j, filelist in enumerate(fileLists):
        for idx, filename in enumerate(filelist):
            clear_output(wait=True)
            print("processing", site,":",termTypeNames[j],": {} of {}".format(idx,len(filelist)-1) )
            try:
                terms = terms + load_list(os.path.join(folder,filename))
            except:
                e = sys.exc_info()
                print(e)
                print(os.path.join(folder,filename))                    
        #write_list(terms, os.path.join("data",site+"_"+termTypeNames[j]+"Terms.txt"))
print("done!")

processing mobafire : header : 2567 of 2568
done!


In [461]:
#re.match("(January)|(February)|(March)|(April)|(May)|(June)|(July)|(August)|(September)|(October)|(November)|(December)] \d+","January 13")
#re.sub(".*/(?=.*$)","","ability/parent/barrel-roll")
#"challenger ahri" in banlist
# replace the -### at the end of page names
re.sub("-\d+$","","celestial-eye-358")




'celestial-eye'

In [43]:
termsFilename = "data/filteredTermsFinal.txt"
terms = load_list(termsFilename)
print(len(terms))

9342


In [108]:
sites=['gamepedia','LeagueFandom','mobafire']

files = list()
for site in sites:        
    #site = sites[0]
    folder = os.path.join("data",site+"Data")
    completedPages = os.path.join("data",site+"_CompletedPagesList.txt")
    completedPagesList = load_list(completedPages)    
    filelist = os.listdir(folder)
    files = files + [os.path.join(folder,b) for b in filelist if "fullText" in b]    

urls = list()
counts = np.zeros([len(files),len(terms)])
countFilename = "data/counts.txt"
with open(countFilename,'w',encoding = 'utf-8',errors='ignore') as f:
    f.write("site\t")
    for t in terms:
        f.write(t+"\t")
    f.write("\n")
for idx, file in enumerate(files):
    #file = files[0]
    #idx = 0
    clear_output(wait=True)
    print(idx, "of",len(files)-1)
    text = ""
    with open(file, 'r', encoding='utf-8',errors='ignore') as f:        
        print(file)
        text = f.read()
        url = re.match(".*\n",text)[0]
        urls.append(url.strip())
        text = re.sub(".*\n","",text,count=1)
        for jdx, t in enumerate(terms):
            count = text.lower().count(t)        
            counts[idx][jdx] = count
    with open(countFilename,'a', encoding = 'utf-8',errors='ignore') as f:
        f.write(urls[idx])
        f.write('\t')
        for c in counts[idx]:
            f.write(str(int(c))+"\t")
        f.write("\n")
#write_list(terms, os.path.join("data",site+"_"+termTypeNames[j]+"Terms.txt"))
write_list(urls,"data/urls.txt")
print("done!")

8003 of 8003
data\mobafireData\zyraguide_fullText.txt
done!


In [4]:
terms = load_list("data/filteredTermsFinal.txt")
urls = load_list("data/urls.txt")
counts = np.zeros([len(urls),len(terms)])
with open("data/counts.txt",'r',encoding='utf-8', errors='ignore') as f:
    idx = 0
    l = f.readline()    
    l = f.readline()
    while l!="":
        clear_output(wait=True)        
        print(idx,"of",len(urls)-1)
        print(urls[idx])
        counts[idx,:] = l.split("\t")[1:-1]
        l = f.readline()
        idx += 1
        
   



8003 of 8004
https://www.mobafire.com/league-of-legends/zyra-guide


In [5]:
# get sum of each term over all sites
wordsums = np.zeros([counts.shape[1]])
badSums = list()
for idx in range(counts.shape[1]):
    c = np.sum(counts[:,idx])
    wordsums[idx] = c    

order = np.argsort(wordsums)

sortedTerms = list()
sortedData = np.zeros(counts.shape)

for idx in range(len(order)):
    o = order[-idx-1]
    sortedTerms.append(terms[o])
    sortedData[:,idx] = counts[:,o]
    


In [181]:
# convert to dataframe and then output to csv
batch_size = 500
s = 0
while s < sortedData.shape[1]:    
    e = s+batch_size
    print(s,"to",e,"of",sortedData.shape[1])
    if e > sortedData.shape[1]:
        e = sortedData.shape[1]
    df = pd.DataFrame(data = sortedData[:,s:e], columns=sortedTerms[s:e])
    df.insert(0,'site',urls)    
    df.to_csv("data/wordCounts_{}_to_{}.txt".format(s,e))    
    s += batch_size
            

0 to 500 of 9342
500 to 1000 of 9342
1000 to 1500 of 9342
1500 to 2000 of 9342
2000 to 2500 of 9342
2500 to 3000 of 9342
3000 to 3500 of 9342
3500 to 4000 of 9342
4000 to 4500 of 9342
4500 to 5000 of 9342
5000 to 5500 of 9342
5500 to 6000 of 9342
6000 to 6500 of 9342
6500 to 7000 of 9342
7000 to 7500 of 9342
7500 to 8000 of 9342
8000 to 8500 of 9342
8500 to 9000 of 9342
9000 to 9500 of 9342


In [100]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
			 "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself",
			 "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which",
			 "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be",
			 "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
			 "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for",
			 "with", "about", "against", "between", "into", "through", "during", "before", "after", "above",
			 "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further",
			 "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
			 "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so",
			 "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]


def filterTerms(terms,banlist=None):
	terms = [b.lower() for b in terms]
	# get the stem from any thing that is directory format
	bonus = list()
	for b in terms:
		if "/" in b:
			bonus += b.split("/")
	terms += bonus		
	terms = [b for b in terms if not "/" in b]
	
	# replace _ with a space
	terms = [re.sub("_"," ", b) for b in terms]
	# replace %27 with a single quote
	terms = [re.sub("%27","\'", b) for b in terms]	
	# remove sentences
	terms = [b for b in terms if len(b.split(" "))<=6]	
	
	# remove things that have no characters in them
	terms = [b for b in terms if re.search("[a-z]+",b)!=None]
	
	
	# uses look-ahead (?=...) and look-behind (?!...) to match [a-z]-[a-z] and replace it with a space
	terms = [re.sub("[-–](?=[a-z])(?!=[a-z])"," ", b) for b in terms]
	
	# remove things that start and end with [],(), and "" or are broken and start/end with only one
	terms = [b for b in terms if re.match("[\(\"\[“].*[\)\"\]”]$",b)==None]
	#remove trailing colon
	terms = [re.sub(":$","",b) for b in terms]
	#terms = [b for b in terms if re.match(".*[\)\"\]”]",b)==None]
	#terms = [b for b in terms if re.match("[\(\"\[“].*",b)==None]
	
	# remove things that start with / and .
	#terms = [b for b in terms if re.match("[\./]",b)==None]
	
	# ---------- delete key phrases section ----------
	
	# remove things like v4.5 and V0.9.8.1
	terms = [b for b in terms if re.match("[vV][\d*\.]+",b)==None]
	# remove Patch xx.xx items
	terms = [b for b in terms if re.match("patch \d*\.\d*",b)==None]	
	
	terms = [re.sub("\(rune\)","",b) for b in terms]
	terms = [re.sub("\(item\)","",b) for b in terms]
	terms = [re.sub("\(legends of runeterra\)","",b) for b in terms]
	# replace the weird single quote with regular single quote
	terms = [b.replace("’","'") for b in terms]	
	# remove FirstName 'nickname' LastName entries
	terms = [b for b in terms if re.match("\D+ \'\S+\' \D+",b)==None]
	
	terms = [b for b in terms if re.match("league of legends v*[0-9]+",b)==None]
	
	
	# remove dates
	terms = [b for b in terms if re.match("(january)|(february)|(march)|(april)|(may)|(june)|(july)|(august)|(september)|(october)|(november)|(december)] \d+",b)==None]
	
    # remove "Passive - "
	terms = [re.sub("passive [-–]","",b) for b in terms]
    
	# delete "Q/W/E/R - Spell Name"
	terms = [re.sub("q [-–] ","", b) for b in terms]
	terms = [re.sub("w [-–] ","", b) for b in terms]
	terms = [re.sub("e [-–] ","", b) for b in terms]
	terms = [re.sub("r [-–] ","", b) for b in terms]
	'''
	# remove +number% of
	terms = [re.sub("\+*\d+%*( of ).*","",b) for b in terms]
	#terms = [re.sub("[(\'s)\.:]$","",b) for b in terms]
	terms = [re.sub("\'$","",b) for b in terms]
    
		
	# remove chromas
	terms = [b for b in terms if re.search("chroma",b)==None]	
	# ---------- final cleanup section ----------
	terms = [b.strip() for b in terms]
	
	if banlist != None:
		terms = [b for b in terms if b not in banlist]
	terms = [b for b in terms if not b in stopwords]	
	'''
	# remove duplicates
	terms = list(dict.fromkeys(terms))
	#terms.sort()
	return terms



In [101]:
l = ['Passive - Deathbringer Stance']
f = filterTerms(l)
print(len(l))
for i in l:
    print(i)
print('-----------')
print(len(f))
for i in f:
    print(i)

1
Passive - Deathbringer Stance
-----------
1
 deathbringer stance


In [114]:
b = 'quinn'
l = re.sub("^[qwer] [-–]","", b)
print(l)

quinn


In [33]:
# load all the terms
print("loading terms from preprocessed files...")
sites = ['gamepedia','LeagueFandom','mobafire']
files = ['_CompletedPagesList.txt','_boldTerms.txt','_headerTerms.txt']	
terms = list()
for site in sites:
	print("loading",site,"...")
	for file in files:		
		filename = os.path.join("data",site+file)
		tmp = load_list(filename)
		terms = terms + tmp
champNames = load_list("championList.txt")
skinNames = load_list("championSkinsList.txt")	
banlist = skinNames
banlist = [b.lower() for b in banlist]
terms.append("Vladimir SEASON 10 mid/top - best vladimir guide for new pla")
t = filterTerms(terms)

['Aatrox',
 'Absolute_Focus_(Rune)',
 'Abyssal_Mask_(Item)',
 'Abyssal_Scepter_(Item)',
 'Adaptive_Helm_(Item)',
 'Aegis_of_the_Legion_(Item)',
 'Aether_Wisp_(Item)',
 'Aftershock_(Rune)',
 'Ahri',
 'Akali']

In [66]:
 re.search("[a-z]+",'0.25')

TypeError: unsupported operand type(s) for &: 'str' and 'int'